#### Stage 3
# Origin Destination Data - Spatial Overview
with folium

##### Generating Flow Maps with OD Data
This notebook contains the code for the generation of visualization **number 7** from **cycleAndFlowODdf.csv**.
The mapping process is similar to those for flow data (see *1S_flows_mapVis.ipynb*) but more filter options are given and normalization is only done with respect to the plotted lines.
The polygons corresponding to the chosen start- and end-cells are coloured white and black.

##### Versions of the used packages:
- pandas: 0.24.2
- numpy: 1.16.4
- folium: 0.11.0
- ipywidgets: 7.5.1

In [1]:
import pandas as pd
from ast import literal_eval
import folium
import numpy as np
import ipywidgets as pyw

In [2]:
centroidBounds = [[[48.04821822889212, 11.67438639612782]],[[48.39922017377597, 11.75558830832081]]]
mapbox_access_token = r'MAPBOX_ACCESS_TOKEN'
tileset_ID_str = 'streets-v11'
# tileset_ID_str = 'outdoors-v11'
# tileset_ID_str = 'light-v10'
# tileset_ID_str = 'dark-v10'
# tileset_ID_str = 'satellite-v9'
# tileset_ID_str = 'satellite-streets-v11'
tilesize_pixels = '256'
#Tiles = 'OpenStreetMap'
#Tiles = f"https://api.mapbox.com/styles/v1/mapbox/{tileset_ID_str}/tiles/{tilesize_pixels}/{{z}}/{{x}}/{{y}}@2x?access_token={mapbox_access_token}"
Tiles = f"https://api.mapbox.com/styles/v1/klavere/ckd8wc3hf06we1imiypydrqcf/tiles/{tilesize_pixels}/{{z}}/{{x}}/{{y}}@2x?access_token={mapbox_access_token}"
MapboxAttribution = '<a href="https://www.mapbox.com/about/maps/">© Mapbox</a> | <a href="http://www.openstreetmap.org/about/">© OpenStreetMap</a> | <a href="https://www.mapbox.com/map-feedback/#/-74.5/40/10">Improve this map</a>'

In [3]:
def csvtodf_SC(path):
    data = pd.read_csv('data/'+path+'.csv',
                       delimiter=';',
                       skipinitialspace=True,
                       skiprows=0)
    df = pd.DataFrame(data)
    return df;

def csvtodf_C(path):
    data = pd.read_csv('data/'+path+'.csv',
                       delimiter=',',
                       skipinitialspace=True,
                       skiprows=0)
    df = pd.DataFrame(data)
    return df;

def getCoordsBack(coords):
    return literal_eval(coords);

def getCoordsListBack(coords):
    newcoordslist = literal_eval(coords)
    newcoordslist = [literal_eval(newcoordslist[0]),literal_eval(newcoordslist[1])]
    return newcoordslist;

In [4]:
celldf = csvtodf_SC('celldf')
celldf['polyCoords'] = celldf.apply(lambda row: getCoordsBack(row.polyCoords), axis = 1)
celldf['centroidCoords'] = celldf.apply(lambda row: getCoordsBack(row.centroidCoords), axis = 1)
celldf = celldf.reindex(columns = ['cellID', 'polyCoords', 'centroidCoords'])
celldf.head()

,cellID,polyCoords,centroidCoords
0,1,"[[48.14605999999999, 11.510794], [48.146254999...","[48.15396752884293, 11.51032914199828]"
1,2,"[[48.23591199999999, 11.635515], [48.237070000...","[48.22263647015427, 11.62774480807693]"
2,3,"[[48.14900599999999, 11.696217], [48.153606000...","[48.13564057367491, 11.70093894926043]"
3,4,"[[48.12879600000001, 11.541556], [48.125282, 1...","[48.13194111905815, 11.54797042679206]"
4,5,"[[48.18833000000001, 11.615821], [48.193665, 1...","[48.19669108307331, 11.6113147037113]"


In [5]:
ODdf = csvtodf_SC('cycleAndFlowODdf')
ODdf['ODCoords'] = ODdf.apply(lambda row: getCoordsListBack(row.ODCoords), axis = 1)
ODdf.head()

,startCellID,endCellID,ODCoords,dayInt,hour,weekday,dayType,Bus,Rail,Tram,...,traveltime_mean_Rail,traveltime_mean_Tram,traveltime_mean_UBahn,traveltime_mean_privat,public,moves,traveltime_mean_public,dist_mean_public,traveltime_mean_moves,dist_mean_moves
0,1,1,"[[48.15396752884293, 11.51032914199828], [48.1...",0,0,Monday,MonThu,2.0,0.0,0.0,...,0.0,0.0,0.0,656.071429,2.0,16.0,251.25,0.06575,453.660714,0.066696
1,1,1,"[[48.15396752884293, 11.51032914199828], [48.1...",0,1,Monday,MonThu,0.0,0.0,0.0,...,0.0,0.0,0.0,556.000000,0.0,8.0,0.00,0.00000,278.000000,0.058250
2,1,1,"[[48.15396752884293, 11.51032914199828], [48.1...",0,2,Monday,MonThu,0.0,0.0,0.0,...,0.0,0.0,0.0,440.225806,0.0,31.0,0.00,0.00000,220.112903,0.064032
3,1,1,"[[48.15396752884293, 11.51032914199828], [48.1...",0,3,Monday,MonThu,0.0,0.0,0.0,...,0.0,0.0,0.0,594.583333,0.0,12.0,0.00,0.00000,297.291667,0.138208
4,1,1,"[[48.15396752884293, 11.51032914199828], [48.1...",0,4,Monday,MonThu,0.0,0.0,0.0,...,0.0,0.0,0.0,598.962963,0.0,27.0,0.00,0.00000,299.481481,0.096148


## constants for mapping function

In [6]:
startCellsStr = str(ODdf.startCellID.unique().copy().tolist()).strip('[]')
endCellsStr = str(ODdf.endCellID.unique().copy().tolist()).strip('[]')

days = ['', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dayTypes = ['', 'MonThu', 'Fri', 'Weekend']

modes = ['moves', 'privat', 'public', 'Rail', 'UBahn', 'Tram', 'Bus']

traveltime_mean_modes = ['']*7
for i in range(0,7):
    traveltime_mean_modes[i] = 'traveltime_mean_'+modes[i]

dist_mean_modes = ['']*7
for i in range(0,7):
    dist_mean_modes[i] = 'dist_mean_'+modes[i]
    
distMax = []
for k in dist_mean_modes:
    distMax.append(max(ODdf[k]))
maxDist = max(distMax)

# cellsN_mean_modes = ['']*7
# for i in range(0,7):
#     cellsN_mean_modes[i] = 'cellsN_mean_'+modes[i]

modesColor = {'privat':'#999999',#'#BFBFBF',
              'Rail':'#4daf4a',
              'UBahn':'#377eb8',
              'Tram':'#e41a1c',
              'Bus':'#984ea3',
              'public':'#ff7f00', # '#ff7f00',
              'moves':'#a65628'} # '#FFFFFF'}
# including ColorBrewer Colors from: https://colorbrewer2.org/#type=qualitative&scheme=Set1&n=9

d = {'ODCoords':['first']}
modesAggMean = dict((key, ['mean']) for key in modes+traveltime_mean_modes+dist_mean_modes)#+cellsN_mean_modes)
modesAggSum = dict((key, ['sum']) for key in modes+traveltime_mean_modes+dist_mean_modes)#+cellsN_mean_modes)
modesAggMixed = dict((key, ['sum']) for key in modes)
modesAggMixed1 = dict((key, ['mean']) for key in traveltime_mean_modes+dist_mean_modes)
modesAggMixed.update(modesAggMixed1)

## mapping function

In [7]:
def mapOD(mode='moves', day='', dayType='',
          traveltime=(0,60), distance=(0,60), 
          StartCells='', EndCells='',
          hours=(18,24), # check maxDist to set limits
          normalize=False):
    
    if(StartCells): startCells = list(map(int, StartCells.split(',')))
    if(EndCells): endCells = list(map(int, EndCells.split(',')))
    
    ttmode = traveltime_mean_modes[modes.index(mode)]
    distmode = dist_mean_modes[modes.index(mode)]
    
    # aggregate flowdf according to filter choices for day, dayType and hour.
    # mean between days, dayTypes, sum over hours for N. mean always for traveltime and distance
    d.update(modesAggMean)
    if (day):
        mapdf = ODdf[(ODdf.weekday == day)].groupby(['startCellID', 'endCellID', 'hour']).agg(d).copy().reset_index()
        mapdf.columns = mapdf.columns.get_level_values(0)
    elif (dayType):
        mapdf = ODdf[(ODdf.dayType == dayType)].groupby(['startCellID', 'endCellID', 'hour']).agg(d).copy().reset_index()
        mapdf.columns = mapdf.columns.get_level_values(0)
    else:
        mapdf = ODdf.groupby(['startCellID', 'endCellID', 'hour']).agg(d).copy().reset_index()
        mapdf.columns = mapdf.columns.get_level_values(0)
        
    # apply startCell and endCell filters
    d.update(modesAggMixed)
    if(StartCells):
        if(EndCells):
            mapdf = mapdf[(mapdf.startCellID != mapdf.endCellID)&
                          (mapdf.startCellID.isin(startCells))&
                          (mapdf.endCellID.isin(endCells))].groupby(['startCellID','endCellID', 'hour']).agg(d).copy().reset_index()
            mapdf.columns = mapdf.columns.get_level_values(0)
        else:
            mapdf = mapdf[(mapdf.startCellID != mapdf.endCellID)&
                          (mapdf.startCellID.isin(startCells))#&(~mapdf.endCellID.isin(startCells))
                          ].groupby(['startCellID','endCellID', 'hour']).agg(d).copy().reset_index()
            mapdf.columns = mapdf.columns.get_level_values(0)
    elif(EndCells):
        mapdf = mapdf[(mapdf.startCellID != mapdf.endCellID)&
                          (mapdf.endCellID.isin(endCells))#&(~mapdf.startCellID.isin(endCells))
                     ].groupby(['startCellID','endCellID', 'hour']).agg(d).copy().reset_index()
        mapdf.columns = mapdf.columns.get_level_values(0)
    else:
        mapdf = mapdf[(mapdf.startCellID != mapdf.endCellID)
                     ].groupby(['startCellID','endCellID', 'hour']).agg(d).copy().reset_index()
        mapdf.columns = mapdf.columns.get_level_values(0)
    
    # apply hour, traveltime and distance filters
    mapdf = mapdf[(mapdf.hour.between(hours[0]-1,hours[1],inclusive=False))&
                  (mapdf[ttmode].between(traveltime[0]*60,traveltime[1]*60,inclusive=True))&
                  (mapdf[distmode].between(distance[0],distance[1],inclusive=True))].groupby(['startCellID',
                                                                                           'endCellID']).agg(d).copy().reset_index()
    mapdf.columns = mapdf.columns.get_level_values(0)

    # initialize map with cell polygons
    m = folium.Map(tiles = Tiles,
                   attr = MapboxAttribution)
    m.fit_bounds(centroidBounds)
    for index, row in celldf.iterrows():
        if (StartCells) and row.cellID in startCells:
            color = '#FFFFFF'
            fillop = 0.5
        elif (EndCells) and row.cellID in endCells:
            color = '#000000'
            fillop = 0.5
        else:
            color = '#ff7f00'
            fillop = 0.02
        line_i = folium.Polygon(locations=row.polyCoords,
                                color = color,#'#ff7f00',#'#FBF9F7',
                                weight = 1,
                                fill = True,
                                opacity = 0.5,
                                fill_opacity = fillop,
                                tooltip = 'Cell'+str(row.cellID)).add_to(m)
    
    # put ODs on map
    color = modesColor[mode]
    mapdf = mapdf[(mapdf[mode] >= 5)] # exclude flows < 5
    M = np.mean(mapdf[mode]) # mean for linewidths if normalized=False
    
    for index, row in mapdf.iterrows():
#         if (normalize):
#             M = row[str('mean'+mode)] # get flowID-specific mean if noramlized=True
#             if (M<5): M = np.nan
#             tooltip = str(int(row[str(mode)]))+' moves<br>'+'mode = '+str(mode)+'<br>'+str(np.round(row[mode]/M, decimals=1))+' times normal amount<br>'+'from cell '+str(row.startCellID)+' to cell '+str(row.startCellID)
#         else: 
        tooltip = str(int(row[str(mode)]))+' moves<br>'+'mode = '+str(mode)+'<br>'+'from cell '+str(row.startCellID)+' to cell '+str(row.endCellID)
        weight = 3*row[mode]/(2*M)
        if (weight):
            line_i = folium.PolyLine(locations = row.ODCoords,
                                     color = color,
                                     weight = weight,
                                     opacity = 0.6,
                                     tooltip = tooltip)
            m.add_child(line_i)
    return m;

## prepare range sliders etc

In [8]:
range_slider_h = pyw.IntRangeSlider(
    value=[0,24],
    min=0, max=24, step=1,
    description='Time of day')

range_slider_tt = pyw.IntRangeSlider(
    value=[0,60],
    min=0, max=60, step=1,
    description='Minutes travelled')

range_slider_td = pyw.IntRangeSlider(
    value=[0,60],
    min=0, max=60, step=1,
    description='km travelled')

days = ['', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dayTypes = ['', 'MonThu', 'Fri', 'Weekend']

StartCell = pyw.SelectMultiple(
    options=celldf['cellID'],
    value=[2,44,71,98],
    #rows=10,
    description='StartCell',
    disabled=False)
EndCell = pyw.SelectMultiple(
    options=celldf['cellID'],
    value=tuple(celldf['cellID']),
    #rows=10,
    description='EndCell',
    disabled=False)

## map ODs

In [9]:
pyw.interact_manual(mapOD,
                    mode = modes,
                    StartCells = '',
                    EndCells = '',
                    day=days, dayType=dayTypes,
                    hours=range_slider_h,
                    traveltime=range_slider_tt,
                    distance=range_slider_td,
                    normalize = pyw.fixed(False))

interactive(children=(Dropdown(description='mode', options=('moves', 'privat', 'public', 'Rail', 'UBahn', 'Tra…

<function __main__.mapOD(mode='moves', day='', dayType='', traveltime=(0, 60), distance=(0, 60), StartCells='', EndCells='', hours=(18, 24), normalize=False)>